In [1]:
import tensorflow as tf
print("TF",tf.__version__)

import time
import numpy as np
import matplotlib.pyplot as plt

import pathlib
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.applications import ResNet152V2,ResNet50V2
from tensorflow.keras.models import Sequential

TF 2.7.0


In [2]:
# data 불러오기
data_dir = 'data_preprocess'
train_dir = os.path.join(data_dir)
train_fnames = os.listdir(train_dir)
NUM_CLASSES = len(train_fnames)
print(NUM_CLASSES)

127


In [3]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                 horizontal_flip = True,
                                  validation_split=0.1)

train_data = train_datagen.flow_from_directory(train_dir,
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode="categorical",
                                              shuffle = True,
                                              subset='training')
validation_data = train_datagen.flow_from_directory(train_dir,
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode="categorical",
                                              shuffle = True,
                                              subset='validation')

Found 23523 images belonging to 127 classes.
Found 2558 images belonging to 127 classes.


In [4]:
earlystopping = EarlyStopping(monitor='val_accuracy', 
                              patience=20,
                              mode = 'max'         
                             )

reduceLR = ReduceLROnPlateau(
    monitor='val_accuracy', 
    factor=0.5,         
    patience=5,         
    min_lr=1e-5
)

In [7]:
def model_learning(Model,model_name,x_data,x_label):
    model = Sequential()
    model = Model
    model.compile(optimizer = Adam(1e-4),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])

    history = model.fit(train_data,
                    epochs = 5,
                    batch_size=128,
                    validation_data = validation_data,
                    callbacks=[reduceLR , earlystopping],
                    )
#################################################################################    
    his_dict = history.history
    loss = his_dict['loss']
    val_loss = his_dict['val_loss'] 

    epochs = range(1, len(loss) + 1)
    fig = plt.figure(figsize = (15, 10))

# 훈련 및 검증 손실 그리기
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(epochs, loss, color = 'blue', label = 'train loss')
    ax1.plot(epochs, val_loss, color = 'red',linestyle='--', label = 'validation loss')
    ax1.set_title('train and val loss')
    ax1.set_xlabel('epochs')
    ax1.set_ylabel('loss')
    ax1.legend()

    acc = his_dict['accuracy']
    val_acc = his_dict['val_accuracy']

# 훈련 및 검증 정확도 그리기
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(epochs, acc, color = 'blue', label = 'train accuracy')
    ax2.plot(epochs, val_acc, color = 'red',linestyle='--', label = 'validation accuracy')
    ax2.set_title('train and val accuracy')
    ax2.set_xlabel('epochs')
    ax2.set_ylabel('accuracy')
    ax2.legend()

    model.save(str(model_name)+".h5")    
    plt.savefig(str(model_name)+str(fold_var)+".png",dpi=200)

    tf.keras.backend.clear_session()

    return